In [1]:
import numpy as np
import cv2
import os

In [2]:
# Load the labels
LABELS = open("/Users/tarensanders/darknet/data/coco.names").read().strip().split("\n")

In [4]:
# Get the weights on config
configpath = "/Users/tarensanders/darknet/cfg/yolov3.cfg"
weightspath = "/Users/tarensanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov3.weights"

net = cv2.dnn.readNetFromDarknet(configpath, weightspath)

In [31]:
# Load the image
testimage = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/1128/1128_day1/1128vid4image00016.jpg"

image = cv2.imread(testimage)
(H,W) = image.shape[:2]

In [32]:
# Determine the output layer names
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [33]:
# Create a blob and run through YOLO
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416,416),swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)

In [34]:
boxes = []
confidences = []
classIDs = []
conf_thresh = 0.5
nms_thresh = 0.3

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        
        if confidence > conf_thresh:
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)            
        
# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf_thresh,nms_thresh)       

In [35]:
for i in idxs.flatten():
    print(LABELS[classIDs[i]])
    print(confidences[i])

tvmonitor
0.649584174156189
person
0.5826625227928162
